In [91]:
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# elem.send_keys(Keys.ENTER)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import re
import time
from datetime import datetime

import pymysql 
import pandas as pd 
import numpy as np
from tqdm import tqdm

In [92]:
class createDB():
    def __init__(self, years=None, genres={'건강취미':55890, '경제경영':170, '고전':2105, '과학':987, 
                                       '사회과학':798, '소설시희곡':1, '에세이':55889, '역사':74, 
                                       '인문학':656, '자기계발':336, '장르소설':112011, '예술대중문화':517}): # '청소년':1137
        if years is None: self.years = [i for i in range(1999, 2023)]
        else: self.years = years
        self.genres = genres
        self.urls, self.dfs = {}, {}
        
    def open_driver(self, url, scroll=False):
        user_agent = 'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        options = webdriver.ChromeOptions()
        options.add_argument(user_agent)
        options.add_argument('headless') # 페이지 안 열기 
        options.add_argument('window-size=1920x1080') # 사이즈 지정
        options.add_argument("disable-gpu") # 안 보이게
        self.driver = webdriver.Chrome(options=options)
        self.driver.implicitly_wait(1)
        self.driver.get(url)
        time.sleep(0.5)
        
        if scroll:
            prev_height = self.driver.execute_script('return document.body.scrollHeight')
            while True:
                self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                time.sleep(1)
                curr_height = self.driver.execute_script('return document.body.scrollHeight')
                if prev_height == curr_height: break
                prev_height = curr_height
        
    def save_urls(self, genre):
        self.urls[genre] = {}
        tqdm_years = tqdm(self.years)
        for year in tqdm_years:
            for i in range(1,7): # 한 페이지 당 50권, 6페이지 반복
                url = 'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=YearlyBest&BranchType=1&CID={}&Year={}&page={}&cnt=300&SortOrder=1'.format(self.genres[genre], year, i)
                self.open_driver(url)
                elems = self.driver.find_elements_by_class_name('bo3')
                for elem in elems:
                    title = elem.find_element_by_tag_name('b').text
                    self.urls[genre][title] = elem.get_attribute('href')
            tqdm_years.set_description(genre+' '+str(year)+' urls')
        tqdm_years.close()

    def connect_mysql(self, connect=True):
        if connect:
            self.db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', 
                                      charset='utf8', cursorclass=pymysql.cursors.DictCursor)
            self.cursor = self.db.cursor()
        else:
            self.db.commit()
            self.db.close
            
    def create_tbl(self, genre):
        self.connect_mysql()
        sql = """CREATE TABLE IF NOT EXISTS best_books_{}(
                title VARCHAR(100), author VARCHAR(100),
                genre_1 VARCHAR(200), genre_2 VARCHAR(300), rate DOUBLE,
                female_10 DOUBLE, male_10 DOUBLE, 
                female_20 DOUBLE, male_20 DOUBLE,
                female_30 DOUBLE, male_30 DOUBLE,
                female_40 DOUBLE, male_40 DOUBLE,
                female_50 DOUBLE, male_50 DOUBLE,
                female_60 DOUBLE, male_60 DOUBLE);""".format(genre)
        self.cursor.execute(sql)
        self.connect_mysql(False)
    
    def fill_tbl(self, genre):
        self.connect_mysql()
        tqdm_book = tqdm(self.urls[genre].values())
        for book_url in tqdm_book:
            try: self.open_driver(book_url, scroll=True)
            except: continue

            title = self.driver.find_element_by_class_name('Ere_bo_title').text
            elem = self.driver.find_elements_by_class_name('Ere_sub2_title')
            author = elem[1].text
            rate = self.driver.find_element(By.XPATH,'//*[@id="wa_product_top1_wa_Top_Ranking_pnlRanking"]/div[2]/a[2]').text
            
            big_cat = self.driver.find_elements_by_css_selector('#ulCategory')
            big_cat_text = [element.text for element in big_cat] 
            cat = [cat_text.split(' > ') for cat_text in big_cat_text[0].split('\n')]
            genre_1 = ', '.join([cat[i][1] for i in range(len(cat))])
            genre_2 = ', '.join([cat[i][2] for i in range(len(cat))])

            action = ActionChains(self.driver)
            action.move_to_element(self.driver.find_element_by_class_name('Ere_prod_mblog_L')).perform()
            self.driver.execute_script('window.scrollBy(-100, -300)')
            per_age = self.driver.find_elements_by_class_name('per')
            per_text = [float(per.text.replace('%','')) for per in per_age[:12]]

            sql = '''INSERT INTO best_books_{0} VALUES(
            "{1}", "{2}", "{3}", "{4}", {5}, {6}, {7}, {8}, {9}, {10}, {11}, {12}, {13}, {14}, {15}, {16}, {17});
            '''.format(genre, title, author, genre_1, genre_2, rate,
                       per_text[0], per_text[1], per_text[2], per_text[3], per_text[4], per_text[5],
                       per_text[6], per_text[7], per_text[8], per_text[9], per_text[10], per_text[11])
            self.cursor.execute(sql)
            self.db.commit()
            tqdm_book.set_description(genre+' insert data')
        self.connect_mysql(False)
        tqdm_book.close()
        
    def get_df(self, genre):
        self.connect_mysql(True)
        sql = 'SELECT * FROM best_books_{}'.format(genre)
        self.cursor.execute(sql)
        self.dfs[genre] = pd.DataFrame(self.cursor.fetchall())
        self.dfs[genre].to_csv('best_books_{}.csv'.format(genre), index=False)
        self.connect_mysql(False)
        print('save best_book_'+genre+'.csv')
    
    def auto(self, genre):
        self.save_urls(genre)
        self.create_tbl(genre)
        self.fill_tbl(genre)
        self.get_df(genre)

In [93]:
years = [i for i in range(2015, 2023)]
myset = createDB(years)
print(myset.years)
print(myset.genres)
# for genre in myset.urls.keys():
#     myset.auto(genre)

[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
{'건강취미': 55890, '경제경영': 170, '고전': 2105, '과학': 987, '사회과학': 798, '소설시희곡': 1, '에세이': 55889, '역사': 74, '인문학': 656, '자기계발': 336, '장르소설': 112011, '예술대중문화': 517}


In [47]:
for genre in myset.genres:
    myset.save_urls(genre)

청소년 2022 urls: 100%|██████████████████████████████████████████████████████████████████| 8/8 [06:18<00:00, 47.32s/it]/s]


In [68]:
import pickle

# # save data
# with open('urls.pickle','wb') as fw:
#     pickle.dump(myset.urls, fw)

# load data
with open('urls.pickle', 'rb') as fr:
    urls = pickle.load(fr)

In [69]:
urls

{'건강취미': {'비밀의 정원': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=44819727',
  '신비의 숲': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=52855250',
  '문구의 모험': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=63822450',
  '비밀의 정원 엽서책': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=49512813',
  '해리 포터 컬러링북': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=69016326',
  '패션소녀 컬러링북': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=51344604',
  '바른 몸이 아름답다': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=56748816',
  '임헌석의 톡톡 건강법': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=18770224',
  '안티 스트레스와 집중력 향상을 위한 점잇기&컬러링북 : 도시편': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=60287540',
  '파리 시크릿': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=48095227',
  '시간의 정원': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=50060686',
  '병원에 가지 말아야 할 81가지 이유': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=19248520',
  '판다언니의 첫 번째 자수': 'https://www

In [94]:
myset.urls = urls
# for genre in myset.genres.keys():
#     myset.create_tbl(genre)
#     myset.fill_tbl(genre)
#     myset.get_df(genre)
#     time.sleep(10)

In [50]:
genre = '건강취미'
# myset.create_tbl(genre)
# myset.fill_tbl(genre)
myset.get_df(genre)

건강취미 2015 urls:  12%|████████                                                        | 1/8 [01:11<08:23, 71.96s/it]t/s]


KeyboardInterrupt: 

In [ ]:
genre = '경제경영'
myset.create_tbl(genre)
myset.fill_tbl(genre)
myset.get_df(genre)

In [ ]:
genre = '고전'
myset.create_tbl(genre)
myset.fill_tbl(genre)
myset.get_df(genre)

In [ ]:
genre = '과학'
myset.create_tbl(genre)
myset.fill_tbl(genre)
myset.get_df(genre)

In [ ]:
genre = '사회과학'
myset.create_tbl(genre)
myset.fill_tbl(genre)
myset.get_df(genre)

In [71]:
genre = '소설시희곡'
myset.create_tbl(genre)
myset.fill_tbl(genre)
myset.get_df(genre)

소설시희곡 insert data:  13%|███████▏                                             | 213/1580 [30:12<3:13:51,  8.51s/it]it/s]


UnexpectedAlertPresentException: Alert Text: 19세 이상만 이용할 수 있는 상품입니다. 로그인해주세요.
Message: unexpected alert open: {Alert text : 19세 이상만 이용할 수 있는 상품입니다. 로그인해주세요.}
  (Session info: headless chrome=114.0.5735.134)


In [ ]:
genre = '에세이'
myset.create_tbl(genre)
myset.fill_tbl(genre)
myset.get_df(genre)

In [ ]:
genre = '역사'
myset.create_tbl(genre)
myset.fill_tbl(genre)
myset.get_df(genre)

In [ ]:
genre = '인문학'
myset.create_tbl(genre)
myset.fill_tbl(genre)
myset.get_df(genre)

In [ ]:
genre = '자기계발'
myset.create_tbl(genre)
myset.fill_tbl(genre)
myset.get_df(genre)

In [ ]:
genre = '장르소설'
myset.create_tbl(genre)
myset.fill_tbl(genre)
myset.get_df(genre)

In [95]:
genre = '예술대중문화'
myset.create_tbl(genre)
myset.fill_tbl(genre)
myset.get_df(genre)

예술대중문화 insert data:  25%|████████████▊                                      | 354/1411 [51:21<2:33:20,  8.70s/it]?it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".Ere_prod_mblog_L"}
  (Session info: headless chrome=114.0.5735.134)


In [96]:
class checkDB():
    def __init__(self, genres=None, years=None):
        if genres is None:
            self.genres = {'건강취미':55890, '경제경영':170, '고전':2105, '과학':987, 
                           '사회과학':798, '소설시희곡':1, '에세이':55889, '역사':74, 
                           '인문학':656, '자기계발':336, '장르소설':112011, '예술대중문화': 517}
            self.years = [i for i in range(2015, 2023)]
        else:
            self.genres, self.years = genres, years
        self.urls, self.dfs = {}, {}
        
    def open_driver(self, url, scroll=False):
        user_agent = 'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        options = webdriver.ChromeOptions()
        options.add_argument(user_agent)
        options.add_argument('headless') # 페이지 안 열기 
        options.add_argument('window-size=1920x1080') # 사이즈 지정
        options.add_argument("disable-gpu") # 안 보이게
        self.driver = webdriver.Chrome(options=options)
        self.driver.implicitly_wait(1)
        self.driver.get(url)
        time.sleep(0.5)
        
        if scroll:
            prev_height = self.driver.execute_script('return document.body.scrollHeight')
            while True:
                self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                time.sleep(1)
                curr_height = self.driver.execute_script('return document.body.scrollHeight')
                if prev_height == curr_height: break
                prev_height = curr_height
        
    def save_urls(self, genre):
        self.urls[genre] = {}
        tqdm_years = tqdm(self.years)
        for year in tqdm_years:
            for i in range(1,7): # 한 페이지 당 50권, 6페이지 반복
                url = 'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=YearlyBest&BranchType=1&CID={}&Year={}&page={}&cnt=300&SortOrder=1'.format(self.genres[genre], year, i)
                self.open_driver(url)
                elems = self.driver.find_elements_by_class_name('bo3')
                for elem in elems:
                    title = elem.find_element_by_tag_name('b').text
                    self.urls[genre][title] = elem.get_attribute('href')
            tqdm_years.set_description(genre+' '+str(year)+' urls')
        tqdm_years.close()

    def connect_mysql(self, connect=True):
        if connect:
            self.db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', 
                                      charset='utf8', cursorclass=pymysql.cursors.DictCursor)
            self.cursor = self.db.cursor()
        else:
            self.db.commit()
            self.db.close
            
    def create_tbl(self, genre):
        self.connect_mysql()
        sql = """CREATE TABLE IF NOT EXISTS best_books_{}(
                title VARCHAR(100), author VARCHAR(100),
                genre_1 VARCHAR(200), genre_2 VARCHAR(300), rate DOUBLE,
                female_10 DOUBLE, male_10 DOUBLE, 
                female_20 DOUBLE, male_20 DOUBLE,
                female_30 DOUBLE, male_30 DOUBLE,
                female_40 DOUBLE, male_40 DOUBLE,
                female_50 DOUBLE, male_50 DOUBLE,
                female_60 DOUBLE, male_60 DOUBLE);""".format(genre)
        self.cursor.execute(sql)
        self.connect_mysql(False)
    
    def fill_tbl(self, genre):
        self.connect_mysql()
        tqdm_book = tqdm(self.urls[genre].values())
        i = 1
        for book_url in tqdm_book:
            if i<=354:
                i += 1
                continue
            try: self.open_driver(book_url, scroll=True)
            except: continue

            title = self.driver.find_element_by_class_name('Ere_bo_title').text
            elem = self.driver.find_elements_by_class_name('Ere_sub2_title')
            author = elem[1].text
            rate = self.driver.find_element(By.XPATH,'//*[@id="wa_product_top1_wa_Top_Ranking_pnlRanking"]/div[2]/a[2]').text

            big_cat = self.driver.find_elements_by_css_selector('#ulCategory')
            big_cat_text = [element.text for element in big_cat] 
            cat = [cat_text.split(' > ') for cat_text in big_cat_text[0].split('\n')]
            genre_1 = ', '.join([cat[i][1] for i in range(len(cat))])
            genre_2 = ', '.join([cat[i][2] for i in range(len(cat))])

            action = ActionChains(self.driver)
            action.move_to_element(self.driver.find_element_by_class_name('Ere_prod_mblog_L')).perform()
            self.driver.execute_script('window.scrollBy(-100, -300)')
            per_age = self.driver.find_elements_by_class_name('per')
            per_text = [float(per.text.replace('%','')) for per in per_age[:12]]

            sql = '''INSERT INTO best_books_{0} VALUES(
            "{1}", "{2}", "{3}", "{4}", {5}, {6}, {7}, {8}, {9}, {10}, {11}, {12}, {13}, {14}, {15}, {16}, {17});
            '''.format(genre, title, author, genre_1, genre_2, rate,
                       per_text[0], per_text[1], per_text[2], per_text[3], per_text[4], per_text[5],
                       per_text[6], per_text[7], per_text[8], per_text[9], per_text[10], per_text[11])
            print(sql)
#             self.cursor.execute(sql)
#             self.db.commit()
            tqdm_book.set_description(genre+' insert data')
            break
        self.connect_mysql(False)
        tqdm_book.close()
        
    def get_df(self, genre):
        self.connect_mysql(True)
        sql = 'SELECT * FROM best_books_{}'.format(genre)
        self.cursor.execute(sql)
        self.dfs[genre] = pd.DataFrame(self.cursor.fetchall())
        self.dfs[genre].to_csv('best_books_{}.csv'.format(genre), index=False)
        self.connect_mysql(False)
        print('save best_book_'+genre+'.csv')
    
    def auto(self, genre):
        self.save_urls(genre)
        self.create_tbl(genre)
        self.fill_tbl(genre)
        self.get_df(genre)

In [97]:
genre = '예술대중문화'
check = checkDB()
check.urls = urls
check.create_tbl(genre)
check.fill_tbl(genre)
check.get_df(genre)

예술대중문화 insert data:  25%|█████████████▎                                       | 354/1411 [00:06<00:19, 53.13it/s]?it/s]

INSERT INTO best_books_예술대중문화 VALUES(
            "킬미 힐미 1", "진수완", "예술/대중문화", "영화/드라마", 9.7, 5.5, 0.0, 43.3, 1.7, 28.7, 1.5, 13.0, 2.8, 1.5, 1.7, 0.4, 0.0);
            
save best_book_예술대중문화.csv


In [2]:
%%time
# genres = ['건강취미','경제경영','고전','과학','사회과학','소설시희곡','에세이',
#         '역사','인문학','자기계발','장르소설','청소년']
genres_mini = {'에세이':170,'경제경영':55889}

user_agent = 'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'

years_mini = [i for i in range(2021,2023)]
urls = {}
for genre, g_id in genres_mini.items():
    urls[genre] = {}
    for year in years_mini:
        for i in range(1,2): # 한 페이지 당 50권, 6페이지 반복
            # 연간 베스트렐러 목록 페이지 불러오기
            url = 'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=YearlyBest&BranchType=1&CID={}&Year={}&page={}&cnt=300&SortOrder=1'.format(g_id, year, i)

            options = webdriver.ChromeOptions()
            options.add_argument(user_agent)
            # 페이지 안 열기
            options.add_argument('headless') 
            options.add_argument('window-size=1920x1080')#사이즈지정
            options.add_argument("disable-gpu")#안보이게
            driver = webdriver.Chrome(options=options)
            driver.implicitly_wait(2)
            driver.get(url)
            time.sleep(1)

            # 각 책의 url 뽑아 urls list에 저장
            elems = driver.find_elements_by_class_name('bo3')
            for j in range(10):
                      #range(len(elems))
                title = elems[j].find_element_by_tag_name('b').text
                urls[genre][title] = elems[j].get_attribute('href')
        print(genre, year)

에세이 2021
에세이 2022
경제경영 2021
경제경영 2022
Wall time: 4min 25s


In [3]:
len(urls['에세이']), len(urls['경제경영'])

(483, 501)

In [7]:
# sql 연결 및 테이블 생성
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()
sql = """
    CREATE TABLE IF NOT EXISTS best_books(
        title VARCHAR(100),
        author VARCHAR(100),
        publisher VARCHAR(100),
        genre_1 VARCHAR(200),
        genre_2 LONGTEXT,
        rate DOUBLE,
        female_10 DOUBLE, male_10 DOUBLE, 
        female_20 DOUBLE, male_20 DOUBLE,
        female_30 DOUBLE, male_30 DOUBLE,
        female_40 DOUBLE, male_40 DOUBLE,
        female_50 DOUBLE, male_50 DOUBLE,
        female_60 DOUBLE, male_60 DOUBLE);"""
cursor.execute(sql)

0

In [29]:
%%time
# 테이블 채우기
for genre, book_dic in urls.items():
    if genre=='경제/경영': continue
    for title, book_url in book_dic.items():

        options = webdriver.ChromeOptions()
        options.add_argument(user_agent)
#         페이지 안 열기
        options.add_argument('headless') 
        options.add_argument('window-size=1920x1080')#사이즈지정
        options.add_argument("disable-gpu")#안보이게
        driver = webdriver.Chrome(options=options)
        driver.maximize_window()
        driver.implicitly_wait(2)
        driver.get(book_url)
        time.sleep(0.5)
        
        title = driver.find_element_by_class_name('Ere_bo_title').text
        elem = driver.find_elements_by_class_name('Ere_sub2_title')
        author, publisher = elem[1].text, elem[2].text
        rate = driver.find_element(By.XPATH,'//*[@id="wa_product_top1_wa_Top_Ranking_pnlRanking"]/div[2]/a[2]').text

        big_cat = driver.find_elements_by_css_selector('#ulCategory')
        big_cat_text = [element.text for element in big_cat] 
        cat = [cat_text.split(' > ') for cat_text in big_cat_text[0].split('\n')]
        genre_1 = ', '.join([cat[i][1] for i in range(len(cat))])
        genre_2 = ', '.join([cat[i][2] for i in range(len(cat))])

        prev_height = driver.execute_script('return document.body.scrollHeight')
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(1)
            curr_height = driver.execute_script('return document.body.scrollHeight')
            if prev_height == curr_height:
                break
            prev_height = curr_height
        action = ActionChains(driver)
        action.move_to_element(driver.find_element_by_class_name('Ere_prod_mblog_L')).perform()
        driver.execute_script('window.scrollBy(-100, -300)')
        per_age = driver.find_elements_by_class_name('per')
        per_text = [float(per.text.replace('%','')) for per in per_age[:12]]
        
        sql = '''INSERT INTO best_books VALUES(
        '{0}', '{1}', '{2}', '{3}', '{4}', {5}, {6}, {7}, {8}, {9}, {10}, {11}, {12}, {13}, {14}, {15}, {16}, {17});
        '''.format(title, author, publisher, genre_1, genre_2, rate,
                   per_text[0], per_text[1], per_text[2], per_text[3], per_text[4], per_text[5],
                   per_text[6], per_text[7], per_text[8], per_text[9], per_text[10], per_text[11])
        cursor.execute(sql)
        print(genre, i)
        i+=1
    print(genre, 'done')
    
db.commit()

에세이 6
에세이 7
에세이 8
에세이 9
에세이 10
에세이 11
에세이 12
에세이 13
에세이 14
에세이 15
에세이 16
에세이 17
에세이 18
에세이 19
에세이 20
에세이 21
에세이 22
에세이 23
에세이 24
에세이 25
에세이 26
에세이 27
에세이 28
에세이 29
에세이 30
에세이 31
에세이 32
에세이 33
에세이 34
에세이 35
에세이 36
에세이 37
에세이 38
에세이 39
에세이 40
에세이 41
에세이 42
에세이 43
에세이 44
에세이 45
에세이 46
에세이 47
에세이 48
에세이 49
에세이 50
에세이 51
에세이 52
에세이 53
에세이 54
에세이 55
에세이 56
에세이 57
에세이 58
에세이 59
에세이 60
에세이 61
에세이 62
에세이 63
에세이 64
에세이 65
에세이 66
에세이 67
에세이 68
에세이 69
에세이 70
에세이 71
에세이 72
에세이 73
에세이 74
에세이 75
에세이 76
에세이 77
에세이 78
에세이 79
에세이 80
에세이 81
에세이 82
에세이 83
에세이 84
에세이 85
에세이 86
에세이 87
에세이 88
에세이 89


Exception ignored in: <function Service.__del__ at 0x000001EAE7F8A790>
Traceback (most recent call last):
  File "C:\Users\Playdata\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 176, in __del__
    self.stop()
  File "C:\Users\Playdata\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 151, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\Playdata\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 132, in send_remote_shutdown_command
    if not self.is_connectable():
  File "C:\Users\Playdata\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 115, in is_connectable
    return utils.is_connectable(self.port)
  File "C:\Users\Playdata\anaconda3\lib\site-packages\selenium\webdriver\common\utils.py", line 106, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
  File "C:\Users\Playdata\anaconda3\lib\socket.py", line 832, in create_connection
    sock.connect(sa)
K

에세이 90
에세이 91
에세이 92


KeyboardInterrupt: 

In [6]:
import sqlite3

In [7]:
# options = webdriver.ChromeOptions()
# options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36')
# driver = webdriver.Chrome(options=options)

# url = 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=292069347'
# driver.get(url)

# prev_height = driver.execute_script('return document.body.scrollHeight')
# while True:
#     driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
#     time.sleep(1)
#     curr_height = driver.execute_script('return document.body.scrollHeight')
#     if prev_height == curr_height:
#         break
#     prev_height = curr_height
        
# from selenium.webdriver import ActionChains
# action = ActionChains(driver)
# action.move_to_element(driver.find_element_by_class_name('Ere_prod_mblog_L')).perform()
# print(1)
# driver.execute_script('window.scrollBy(-100, -300)')
# print(2)
# per_age = driver.find_elements_by_class_name('per')
# per_text = [float(per.text.replace('%','')) for per in per_age[:12]]
# print(per_text)